In [1]:
import pandas as pd

In [2]:
overdrive_fh  = "./reports/BPL/od-bpl-all.csv"

## Overdrive bulk MarcExpress dataframe

In [3]:
odf = pd.read_csv(overdrive_fh, header=None, names=["overdriveNo", "overdriveControlNo", "sierraFormat_ext"])

In [8]:
odf.shape

(217134, 3)

In [4]:
odf.head()

,overdriveNo,overdriveControlNo,sierraFormat_ext
0,0E90D7A5-30B8-4D07-9D13-DF0E02EA631E,ODN0000082587,z
1,C449DE3C-A683-412A-8997-F0551A5DC576,ODN0000082663,z
2,765A1B6A-E978-4777-8DD1-9647D611A58B,ODN0000082589,z
3,0957DA4E-DD2F-469E-BDFF-B9B1A9422C21,ODN0000082590,z
4,E572EC2C-09F7-462D-8153-7C8A7493586F,ODN0000082591,z


In [5]:
# find if any missing IDs
odf_missing_ids = odf[odf["overdriveNo"].isnull()]

In [6]:
odf_missing_ids.shape

(0, 3)

In [7]:
odf["sierraFormat_ext"].value_counts()

x    171343
z     43806
v      1985
Name: sierraFormat_ext, dtype: int64

## Sierra Overdrive bibs dataframe

In [10]:
sierra_fh = "./reports/BPL/sierra-bpl-all.csv"

In [11]:
sdf = pd.read_csv(
    sierra_fh,
    header=None,
    names=["overdriveNo", "overdriveNoSrc", "bibNo", "sierraControlNo", "sierraControlNoSrc", "sierraFormat_int", "sierra_status"],
    low_memory=False
)

In [15]:
# this will be larger than number of actual records since each instance of overdriveNo appearing on a bib is recorded in its own row, sometimes as duplicate one in 037, one in 856
sdf.shape

(300410, 7)

In [16]:
sdf.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-
1,27C3C666-343B-442B-A81C-2068E6D1A664,url,b112402306,ocm54908102,OCoLC,x,-
2,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-
3,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,url,b112402318,ocm55680353,OCoLC,x,-
4,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-


In [18]:
sdf["overdriveNoSrc"].value_counts()

037    197866
url    102544
Name: overdriveNoSrc, dtype: int64

In [11]:
sdf["sierraFormat_int"].value_counts()

x    161347
z     41264
v       935
s        45
Name: sierraFormat_int, dtype: int64

In [19]:
# sierra fomat "s"?
sdf_s_format = sdf[sdf["sierraFormat_int"] == "s"]

In [20]:
sdf_s_format.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
1162,15B32E52-357A-4D74-A43E-18C673B0B0FC,url,b112467854,ocm57507474,OCoLC,s,-
1163,EBC278AA-D4EA-45FB-8B3B-DFDF2122ADA0,url,b112467866,ocm57507588,OCoLC,s,-
1164,35AC0D13-E37C-47E0-8DC9-DEEFC69D10DF,url,b112467878,ocm57507915,OCoLC,s,-
1166,7B5DF2A5-6C03-495A-B98B-4A6B49D7BF6A,url,b112467891,ocm57508902,OCoLC,s,-
1167,679C31DB-8F49-4A59-B697-C0CE35CDD490,url,b112467908,ocm57509118,OCoLC,s,-


#### Sierra bibs missing proper Overdrive ID

In [23]:
sdf_no_overdriveNo = sdf[(sdf["overdriveNo"].isnull())|(sdf["overdriveNo"] == "")]

In [24]:
sdf_no_overdriveNo.shape

(0, 7)

In [ ]:
# the result above is unexpected - it seems my multi parsing found overdriveNo in each record

In [25]:
sdf_invalid_overdriveNo = sdf[(sdf["overdriveNo"].isnull())|(sdf["overdriveNo"].str.match("(?!.*-.*-.*.*)"))]

In [26]:
sdf_invalid_overdriveNo.shape

(0, 7)

## Inner join between Sierra and bulk MarcExpress - records already in the catalog

In [37]:
# this looks like legitimate bibs that are marked differently in Sierra than on Overload platform

In [38]:
df_available = pd.merge(sdf, odf, on="overdriveNo", how="inner")

In [39]:
df_available.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x
1,27C3C666-343B-442B-A81C-2068E6D1A664,url,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x
2,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x
3,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,url,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x
4,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-,ODN0000060618,x


In [40]:
df_available.shape

(278211, 9)

In [41]:
# but the results above includes duplicate bibs (multi overdriveNo!); need to dedup to get correct count

In [42]:
df_found_deduped_on_bibNo = df_available.drop_duplicates(subset="bibNo", keep="first").copy()

In [43]:
df_found_deduped_on_bibNo.shape

(189832, 9)

In [44]:
df_found_deduped_on_overdriveNo = df_available.drop_duplicates(subset="overdriveNo", keep="first").copy()

In [45]:
df_found_deduped_on_overdriveNo.shape

(187539, 9)

In [ ]:
# why discrepancy in numbers?
# only logical explanation is because we have duplicates in Sierra!

In [46]:
df_found_deduped_on_bibNo.to_csv("./reports/BPL/found-in-catalog.csv", index=False)

In [47]:
df_found_deduped_on_overdriveNo.to_csv("./reports/BPL/found-in-catalog-unique-overdriveNos.csv", index=False)

#### Suppressed/marked for deletion bibs in Sierra to be reinstated

In [48]:
# find records that are in the catalog, users have access to, but are suppressed
df_found_deduped_on_bibNo["sierra_status"].value_counts()

-    189683
n       127
d        22
Name: sierra_status, dtype: int64

In [49]:
df_reinstate_access = df_found_deduped_on_bibNo[(df_found_deduped_on_bibNo["sierra_status"]=="n")|(df_found_deduped_on_bibNo["sierra_status"]=="d")]

In [50]:
df_reinstate_access.shape

(149, 9)

In [51]:
df_reinstate_access.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext
49,97ABE6CD-7D83-475B-8278-210FB71B35DC,url,b112411459,ocm57707762,OCoLC,z,n,ODN0000062189,z
651,E48D1C32-FDC5-4F77-BCF4-33A62F63B139,037,b112414916,ocm56554714,OCoLC,x,n,ODN0000056568,x
1069,1C0B9DC4-653A-49F7-9187-2176F6BA859D,url,b112467064,ocm57398773,OCoLC,z,n,ODN0000060795,z
2785,E9C9FF5A-3CB1-479A-846C-17CAA8097145,url,b113773043,ocm76747399,OCoLC,v,n,ODN0000108691,v
2887,3646A268-2CCF-4A88-B793-65CD7071C44F,url,b113790211,ocm81284258,OCoLC,z,n,ODN0000114416,z


In [52]:
df_reinstate_access.to_csv("./reports/BPL/reinstate_access-alldata.csv", index=False)

In [53]:
df_reinstate_access["bibNo"].to_csv("./reports/BPL/reinstate_access-bibNos", index=False)

## Missing from Sierra bibs / Sierra and MarcExpress right anti join

In [54]:
mdf_join = pd.merge(sdf, odf, how="outer", on="overdriveNo", indicator=True)

In [55]:
mdf_join.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext,_merge
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x,both
1,27C3C666-343B-442B-A81C-2068E6D1A664,url,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x,both
2,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x,both
3,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,url,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x,both
4,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-,ODN0000060618,x,both


In [56]:
mdf_join.shape

(330005, 10)

In [57]:
df_missing = mdf_join[mdf_join["_merge"] == "right_only"]

In [58]:
df_missing.shape

(29595, 10)

In [59]:
df_missing.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext,_merge
300410,D4B34CBB-80AC-4B53-9CE1-7019929155BA,NaN,NaN,NaN,NaN,NaN,NaN,ODN0000637972,z,right_only
300411,35B73FE2-F9D9-463D-A41E-1EBF6C85F834,NaN,NaN,NaN,NaN,NaN,NaN,ODN0000213530,z,right_only
300412,A4324F25-7F12-4488-8378-BD26C1B2CD34,NaN,NaN,NaN,NaN,NaN,NaN,ODN0000644843,z,right_only
300413,E236C55B-D2F5-4E7F-B832-1B6E58205DA0,NaN,NaN,NaN,NaN,NaN,NaN,ODN0000637978,z,right_only
300414,722370E6-1A28-444E-B58F-A21D5AD05C8B,NaN,NaN,NaN,NaN,NaN,NaN,ODN0000644847,z,right_only


In [60]:
df_missing["sierraFormat_ext"].value_counts()

x    25582
z     2927
v     1086
Name: sierraFormat_ext, dtype: int64

In [61]:
df_missing.to_csv("./reports/BPL/missing.csv", index=False)

In [62]:
df_missing["overdriveNo"].to_csv("./reports/BPL/missing-overdriveNos.csv", index=False)

## overdriveNo not found in bulk MarcExpress records / left anti join

In [63]:
# do not include bibs without proper overdrive #

In [65]:
vsdf = sdf[(sdf["overdriveNo"].notnull())|(sdf["overdriveNo"].str.match("(.*-.*-.*-.*)"))]

In [66]:
vsdf.shape

(300410, 7)

In [67]:
vsdf.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-
1,27C3C666-343B-442B-A81C-2068E6D1A664,url,b112402306,ocm54908102,OCoLC,x,-
2,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-
3,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,url,b112402318,ocm55680353,OCoLC,x,-
4,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-


In [69]:
edf_join = pd.merge(vsdf, odf, how="outer", on="overdriveNo", indicator=True)

In [70]:
edf_join.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext,_merge
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x,both
1,27C3C666-343B-442B-A81C-2068E6D1A664,url,b112402306,ocm54908102,OCoLC,x,-,ODN0000052705,x,both
2,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x,both
3,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,url,b112402318,ocm55680353,OCoLC,x,-,ODN0000055232,x,both
4,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-,ODN0000060618,x,both


In [71]:
edf = edf_join[edf_join["_merge"] == "left_only"]

In [72]:
edf.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status,overdriveControlNo,sierraFormat_ext,_merge
34,07F268B6-EF5B-4953-BBB7-773FEE0C8176,url,b112411277,ocm57506094,OCoLC,z,-,NaN,NaN,left_only
38,AC7F7BAB-FB41-4C1B-8342-852CA2D05309,037,b112411307,ocm57675482,OCoLC,z,-,NaN,NaN,left_only
39,AC7F7BAB-FB41-4C1B-8342-852CA2D05309,url,b112411307,ocm57675482,OCoLC,z,-,NaN,NaN,left_only
43,72023498-8893-464A-B55A-F7CBF0917F22,url,b112411332,ocm57687593,OCoLC,z,-,NaN,NaN,left_only
45,D80763AA-CA4F-42BD-85DA-EB0E65552533,url,b112411356,ocm57696112,OCoLC,z,-,NaN,NaN,left_only


In [73]:
edf.shape

(22199, 10)

In [74]:
# but these rows includes multi overdriveNo on bib, so it needs to be compared to what was found to be overlaping (inner join); overdriveNo are irrelevant here
# lets dedup on bibNo this set

In [75]:
edf_deduped = edf.drop_duplicates(subset="bibNo", keep="first")

In [76]:
edf_deduped.shape

(14624, 10)

In [79]:
# lets see if any of these show up in the inner join; should be 0 overlap

In [81]:
vedf = pd.merge(df_found_deduped_on_bibNo, edf_deduped, on="overdriveNo", how="inner")

In [82]:
vedf.shape

(0, 18)

In [ ]:
# Yay! but ...
# this must be verfied again against Overdrive platform to confirm the library has no access and can be deleted

In [77]:
edf_deduped.to_csv("./reports/BPL/expired-before-verfication.csv", index=False)

In [78]:
edf_deduped[["overdriveNo", "bibNo"]].to_csv("./reports/BPL/expired-before-verification-idsonly.csv", index=False)

## Missing (left anti join) ISBN/control number verification

In [86]:
svdf = pd.read_csv("./reports/BPL/sierra-ids.csv", header=None, names=["isbn", "controlNo_int", "bibNo", "sierraFormat_int", "sierra_status"])

In [87]:
svdf.shape

(439255, 5)

In [88]:
svdf.head()

,isbn,controlNo_int,bibNo,sierraFormat_int,sierra_status
0,079532782x,ocm54908102,b112402306,x,-
1,0795328524,ocm54908102,b112402306,x,-
2,1595472797,ocm55680353,b112402318,x,-
3,0759513740,ocm60397130,b112402343,x,-
4,0060569743,ocm52513883,b112412221,x,-


In [92]:
mvdf = pd.read_csv("./reports/BPL/missing-combined-ids.csv", header=None, names=["isbn", "controlNo_ext", "overdriveNo", "sierraFormat_ext"])

In [93]:
mvdf.shape

(29941, 4)

In [94]:
# size is bigger than missing because of multi isbns on few records

In [95]:
mvdf.head()

,isbn,controlNo_ext,overdriveNo,sierraFormat_ext
0,9781445867519,odn0000637972,D4B34CBB-80AC-4B53-9CE1-7019929155BA,z
1,9781408401910,odn0000213530,35B73FE2-F9D9-463D-A41E-1EBF6C85F834,z
2,9781445816388,odn0000644843,A4324F25-7F12-4488-8378-BD26C1B2CD34,z
3,9781445867496,odn0000637978,E236C55B-D2F5-4E7F-B832-1B6E58205DA0,z
4,9781445861555,odn0000644847,722370E6-1A28-444E-B58F-A21D5AD05C8B,z


#### find inner join if exists on isbn

In [96]:
inner_isbn_df = pd.merge(svdf, mvdf, how="inner", on="isbn")

In [97]:
inner_isbn_df.shape

(109, 8)

In [98]:
inner_isbn_df.head()

,isbn,controlNo_int,bibNo,sierraFormat_int,sierra_status,controlNo_ext,overdriveNo,sierraFormat_ext
0,9781426800610,ocn124311732,b113771319,x,-,odn0000124723,E6B8CE5F-5DE7-4DB9-A069-4FCD20C5721E,x
1,9781426800610,ocn124311732,b113771319,x,-,odn0000124723,E6B8CE5F-5DE7-4DB9-A069-4FCD20C5721E,x
2,9781101098226,ocn458558405,b115056592,x,-,odn0000612090,7B8B4CAC-8C69-406E-90AE-A50E6B601288,x
3,9781101046340,ocn459783191,b115130718,x,-,odn0000675399,CD17127A-7A82-44D4-86B4-D09DD1769C90,x
4,9781101076156,ocn492100143,b11523620x,x,-,odn0000612042,96933AB2-AD0E-4667-9555-7F326CE6D334,x


In [99]:
inner_isbn_df.to_csv("./reports/BPL/found-in-catalog-isbn-match.csv", index=False)

#### find inner join on control number

In [101]:
inner_control_df = pd.merge(svdf, mvdf, how="inner", left_on="controlNo_int", right_on="controlNo_ext")

In [102]:
inner_control_df.shape

(0, 9)

#### left anti join for true missing

In [103]:
# since there is no overloap on controlNo, no need to worry about it on the left anti join 

In [111]:
mjoin_df = pd.merge(svdf, mvdf, how="outer", on="isbn", indicator=True)

In [112]:
mjoin_df.shape

(469090, 9)

In [113]:
mjoin_df.head()

,isbn,controlNo_int,bibNo,sierraFormat_int,sierra_status,controlNo_ext,overdriveNo,sierraFormat_ext,_merge
0,079532782x,ocm54908102,b112402306,x,-,NaN,NaN,NaN,left_only
1,0795328524,ocm54908102,b112402306,x,-,NaN,NaN,NaN,left_only
2,1595472797,ocm55680353,b112402318,x,-,NaN,NaN,NaN,left_only
3,0759513740,ocm60397130,b112402343,x,-,NaN,NaN,NaN,left_only
4,0060569743,ocm52513883,b112412221,x,-,NaN,NaN,NaN,left_only


In [114]:
right_only_df = mjoin_df[mjoin_df["_merge"] == "right_only"]

In [115]:
right_only_df.shape

(29835, 9)

In [116]:
right_only_df.head()

,isbn,controlNo_int,bibNo,sierraFormat_int,sierra_status,controlNo_ext,overdriveNo,sierraFormat_ext,_merge
439255,9781445867519,NaN,NaN,NaN,NaN,odn0000637972,D4B34CBB-80AC-4B53-9CE1-7019929155BA,z,right_only
439256,9781408401910,NaN,NaN,NaN,NaN,odn0000213530,35B73FE2-F9D9-463D-A41E-1EBF6C85F834,z,right_only
439257,9781445816388,NaN,NaN,NaN,NaN,odn0000644843,A4324F25-7F12-4488-8378-BD26C1B2CD34,z,right_only
439258,9781445867496,NaN,NaN,NaN,NaN,odn0000637978,E236C55B-D2F5-4E7F-B832-1B6E58205DA0,z,right_only
439259,9781445861555,NaN,NaN,NaN,NaN,odn0000644847,722370E6-1A28-444E-B58F-A21D5AD05C8B,z,right_only


In [110]:
# dedup time

In [118]:
missing_verified_df = right_only_df.drop_duplicates(subset="overdriveNo", keep="first")

In [119]:
missing_verified_df.shape

(29491, 9)

In [121]:
missing_verified_df.to_csv("./reports/BPL/missing-verified.csv", index=False)